In [1]:
###Flickr
##From https://gist.github.com/DanielBeckstein/1315d96dc52b0692dce1057e57c1e8f5

In [2]:
from __future__ import unicode_literals

import requests
import json
import os
import errno

import urllib

In [3]:

# you might want to moving this function into a seperate util (util.py)
def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

# you might want to moving this class into a seperate file (FlickrApiWrapper.py)
class FlickrApiWrapper():
    """
        Documentation of Flickr API:
        https://www.flickr.com/services/api/
    """
    def __init__(self):
    
        self.dir_json_info = "json_info"
        # create this dir
        mkdir(self.dir_json_info)
        
        self.url_base = "https://api.flickr.com/services/rest/"
        self.url_api = self.url_base
        
        self.api_key = self.get_api_key()
        self.extras = self.get_default_extras()
        self.format = "json"
        
        # downloads the names of all available licenses from flickr into a json file
        self.helper_func_download_stored_info_licenses()
        
    def helper_func_download_stored_info_licenses(self):
        # this function is not very clean,
        params = {}
        params.update( {"method":"flickr.photos.licenses.getInfo"} )
        
        r = self.search(params)
        print "url", r.url
        content = r.text
        jso = json.loads(content.decode("utf-8"))
        js = json.dumps(jso, indent=4, sort_keys=True) # for flickr
        
        with open(self.dir_json_info+"/"+"stored_info_licenses.json","w") as f:
            f.write(js.encode("utf8"))
        
    # prints a json object formatted nicely
    def print_nice(self, jso):
        print json.dumps(jso, indent=4, sort_keys=True)
        
    def get_api_key(self):
        """
        with open(self.dir_json_info+"/"+"api_keys.json","r") as f:
            api_keys = json.load(f)
        key = api_keys["flickr"]["key"]
        """
        key = " . . . "
        #raise ValueError("put your key into this code (line above), then delete this line")
        #how to get your api key: https://www.flickr.com/services/apps/create/apply/
        
        return key
        
    def get_default_extras(self):
               
        extras = []
        extras.append( "views"  )
        extras.append( "license"  )
        #extras.append( "original_format"  )
        extras.append( "tags"  )
        #extras.append( "machine_tags"  )
        extras.append( "owner_name"  )
        extras.append( "url_s"  )
        extras.append( "url_m"  )
        extras.append( "url_l"  )
        extras.append( "url_q"  )
        extras.append( "url_o"  )
        #extras.append( "description"  )

        # list of possible values
        e = """description, license, date_upload, date_taken, owner_name, icon_server, 
        original_format, last_update, geo, tags, machine_tags, 
        o_dims, views, media, path_alias, url_sq, url_t, url_s, url_q, url_m, url_n, url_z, url_c, url_l, url_o"""
        
        return extras
        
    def get_info_license_by_id(self, id):
    
        id_str = str(id)
        if id_str.isdigit():
            id_int = int(id)
            if not ( id_int >= 0 and id_int <= 10 ):
                raise ValueError("id is not in range [0...10]")
            id_str = str(id_int)
        else:
            raise ValueError("id is not integer or digit formatted as string")
        
        with open(self.dir_json_info+"/"+"stored_info_licenses.json","r") as f:
            json_licenses = json.load(f)
        
        licenses = json_licenses["licenses"]["license"]
        license = None
        for l in licenses:
            if l["id"] == id_str:
                license = l
                break
        return license
   
    def search(self, params):
    
        params.update( {"format":self.format} )
        params.update( {"api_key":self.api_key} )
        params.update( {"extras": ",".join(self.extras)} )        
        # nojsoncallback fixes weird invalid json - objects in the response from twitter
        params.update( {"nojsoncallback":"1"} )
        
        return requests.get(self.url_api, params=params)



In [4]:
def mkdir_all(path, stop_from_end_splits=1):
    path_parts = path.split("/")
    print "parts"
    print path_parts
    msg = []
    for i, _ in enumerate( path_parts ):
        if i < len(path_parts) - stop_from_end_splits:
            path_curr = "/".join( path_parts[:i+1] )
            msg += [ ("path", "mkdir_all", path_curr) ]
            mkdir(path_curr)
    print "__mkdir_all__", msg[-1][2]

def mkdir(path):
        try:
            os.makedirs(path)
        except OSError as exc:  # Python >2.5
            if exc.errno == errno.EEXIST and os.path.isdir(path):
                pass
            else:
                raise
        

def fetch_img_form_url(url, path_out):

    url_parts = url.split("/")
    fname = url_parts[-1]
    
    path_out = path_out + fname
    
    if 1:
        print "url", url
        print "fname", fname
        print "path_out", path_out
        
    urllib.urlretrieve(url, path_out)

In [5]:
flag_search_by_text = False

flickr = FlickrApiWrapper()

params = {}

# get info about licenses
if 0:
    params.update( {"method":"flickr.photos.licenses.getInfo"} )

# get a list of public photos
if 0:
    params.update( {"method":"flickr.people.getPublicPhotos"} )
    params.update( {"user_id":"112616056@N08"} )

# get sizes of photo
if 1:
    params.update( {"method":"flickr.photos.getSizes"} )
    params.update( {"photo_id":"31083960980"} )   

# get info of photo
if 0:
    params.update( {"method":"flickr.photos.getInfo"} )
    params.update( {"photo_id":"31083960980"} )   

# search by text
if 1:
    flag_search_by_text = True
    params.update( {"method":"flickr.photos.search"} )
    #https://www.flickr.com/services/api/flickr.photos.licenses.getInfo.html

    # limit photos to certain licenses
    params.update( {"license":"4,5,6,7,8,9,10"} )    
    params.update( {"text":"bulldozer"} )
    params.update( {"per_page":"10"} )
    """        
    date-posted-asc, date-posted-desc, date-taken-asc, date-taken-desc
    interestingness-desc, interestingness-asc, and relevance.
    """        
    params.update( {"sort":"relevance"} )    
    #params.update( {"sort":"interestingness-desc"} )    

r = flickr.search(params)

print "url", r.url

content = r.text
print content.decode("utf-8")[:70]

with open(flickr.dir_json_info+"/"+"response.html","w") as f: #xml
    f.write(content)

#content_formatted = content.replace( "jsonFlickrApi(","")
#content_formatted = content_formatted.replace( "})","}")
content_formatted = content

jso = json.loads(content_formatted.decode("utf-8"))

# log and print
js = json.dumps(jso, indent=4, sort_keys=True) # for flickr
with open(flickr.dir_json_info+"/"+"json_response.json","w") as f:
    f.write(js.encode("utf8"))

print js[:400]
######################


url https://api.flickr.com/services/rest/?api_key=105cca110560523906e6f502ce002ae0&nojsoncallback=1&method=flickr.photos.licenses.getInfo&extras=views%2Clicense%2Ctags%2Cowner_name%2Curl_s%2Curl_m%2Curl_l%2Curl_q%2Curl_o&format=json
url https://api.flickr.com/services/rest/?sort=relevance&api_key=105cca110560523906e6f502ce002ae0&photo_id=31083960980&license=4%2C5%2C6%2C7%2C8%2C9%2C10&per_page=10&text=bulldozer&format=json&nojsoncallback=1&method=flickr.photos.search&extras=views%2Clicense%2Ctags%2Cowner_name%2Curl_s%2Curl_m%2Curl_l%2Curl_q%2Curl_o
{"photos":{"page":1,"pages":252,"perpage":10,"total":"2518","photo":[{
{
    "photos": {
        "page": 1, 
        "pages": 252, 
        "perpage": 10, 
        "photo": [
            {
                "farm": 3, 
                "height_l": "683", 
                "height_m": "333", 
                "height_o": "4000", 
                "height_q": "150", 
                "height_s": "160", 
                "id": "14686561903", 
          

In [6]:
photos = jso["photos"]["photo"]

In [7]:
path_output_folder = "imgs/"
    
print "STATUS - storing images in", path_output_folder
    
# makes sure path_out gets created if not exists
# this is needed in python
mkdir(path_output_folder)

STATUS - storing images in imgs/


In [8]:
if flag_search_by_text:

    flickr.print_nice(jso["photos"])

    print "showing per_page", jso["photos"]["perpage"]
    print "total photos found", jso["photos"]["total"]
    photos = jso["photos"]["photo"]

    for p in photos:
        #flickr.print_nice( p )
        print p["ownername"]
        print p["tags"]
        print p["title"]
        print p["views"]
        print p["height_s"]
        print p["url_l"]
        print flickr.get_info_license_by_id(p["license"])
        fetch_img_form_url(p["url_l"], path_output_folder)
        #print p["url_m"]
    #return photos

if 0:
    license_info = flickr.get_info_license_by_id(2)
    print license_info

# bug - you might want to return something
 #   return None


{
    "page": 1, 
    "pages": 252, 
    "perpage": 10, 
    "photo": [
        {
            "farm": 3, 
            "height_l": "683", 
            "height_m": "333", 
            "height_o": "4000", 
            "height_q": "150", 
            "height_s": "160", 
            "id": "14686561903", 
            "isfamily": 0, 
            "isfriend": 0, 
            "ispublic": 1, 
            "license": "5", 
            "owner": "61338164@N02", 
            "ownername": "haru__q", 
            "secret": "1b4ab018f8", 
            "server": "2911", 
            "tags": "sony a7 bulldozer summaron leitz \u30d6\u30eb\u30c9\u30fc\u30b6\u30fc", 
            "title": "Bulldozer", 
            "url_l": "https://farm3.staticflickr.com/2911/14686561903_1b4ab018f8_b.jpg", 
            "url_m": "https://farm3.staticflickr.com/2911/14686561903_1b4ab018f8.jpg", 
            "url_o": "https://farm3.staticflickr.com/2911/14686561903_cdf7aeff42_o.jpg", 
            "url_q": "https://farm3.staticflic

KeyError: u'url_l'